## Pricing and Market Data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sovai as sov

sov.token_auth(token="your_token")

#### Processed Dataset

In [3]:
tickers = ["MSFT","TSLA", "AMZN", "DDD", "JPM","GS","F","A"]
df_price = sov.data("market/prices", tickers)

In [4]:
df_price.tail()

open    high     low   close       volume  closeadj  \
ticker date                                                                
TSLA   2024-08-06 200.750 202.900 192.670 200.640 72794376.000   200.640   
       2024-08-07 200.770 203.490 191.480 191.760 69242272.000   191.760   
       2024-08-08 195.700 200.700 192.040 198.840 64076848.000   198.840   
       2024-08-09 197.050 200.880 195.110 200.000 57959436.000   200.000   
       2024-08-12 199.020 199.260 194.670 197.490 62136624.000   197.490   

                   closeunadj  
ticker date                    
TSLA   2024-08-06     200.640  
       2024-08-07     191.760  
       2024-08-08     198.840  
       2024-08-09     200.000  
       2024-08-12     197.490

#### Grab just closing prices

In [5]:
df_close = sov.data("market/closeadj", tickers, purge_cache=True)

In [6]:
df_close

closeadj
ticker date                
A      1999-11-18    26.917
       1999-11-19    24.703
       1999-11-22    26.917
       1999-11-23    24.623
       1999-11-24    25.119
...                     ...
TSLA   2024-08-06   200.640
       2024-08-07   191.760
       2024-08-08   198.840
       2024-08-09   200.000
       2024-08-12   197.490

[48711 rows x 1 columns]

Let's add some technical indicators

In [11]:
df_tech = df_price.technical_indicators()

In [12]:
df_tech.tail()

open    high     low   close       volume  closeadj  \
ticker date                                                                
TSLA   2024-08-06 200.750 202.900 192.670 200.640 72794376.000   200.640   
       2024-08-07 200.770 203.490 191.480 191.760 69242272.000   191.760   
       2024-08-08 195.700 200.700 192.040 198.840 64076848.000   198.840   
       2024-08-09 197.050 200.880 195.110 200.000 57959436.000   200.000   
       2024-08-12 199.020 199.260 194.670 197.490 62136624.000   197.490   

                   closeunadj  ADX_14  CCI_14  MFI_14  WILLR_14  \
ticker date                                                       
TSLA   2024-08-06     200.640 100.000  92.266  88.487  -100.000   
       2024-08-07     191.760 100.000 -46.788  88.487  -100.000   
       2024-08-08     198.840 100.000 -45.507  88.487  -100.000   
       2024-08-09     200.000 100.000  -3.608  88.487  -100.000   
       2024-08-12     197.490 100.000  -5.777  88.487  -100.000   

                                                              ATR_14    BOP  \
ticker date                                                                   
TSLA   2024-08-06 15713699181293156529539840620523563754399903939... -0.000   
       2024-08-07 14591652854383321310700681324873716827277449661... -0.000   
       2024-08-08 13550305675896316432581325270852071425719941095... -0.000   
       2024-08-09 12606628872168773472315472701788157776094965101... -0.000   
       2024-08-12 11730665542246673909564430501221857161333991147... -0.000   

                   CMO_14   DX_14  KAMA_30    PPO  ROCR_10  TRIX_30  ULTOSC  \
ticker date                                                                   
TSLA   2024-08-06   0.000 100.000    0.000 42.366    0.000   -6.347   0.000   
       2024-08-07   0.000 100.000    0.000 42.366    0.000   -6.347   0.000   
       2024-08-08   0.000 100.000    0.000 42.366    0.000   -6.347   0.000   
       2024-08-09   0.000 100.000    0.000 42.366    0.000   -6.347   0.000   
       2024-08-12   0.000 100.000    0.000 42.366    0.000   -6.347   0.000   

                   NATR_14  HT_DCPERIOD  HT_DCPHASE  HT_TRENDMODE  \
ticker date                                                         
TSLA   2024-08-06    0.000       44.283     184.039             1   
       2024-08-07    0.000       42.483     184.188             1   
       2024-08-08    0.000       40.432     184.404             1   
       2024-08-09    0.000       39.964     184.508             1   
       2024-08-12    0.000       40.376     184.416             1   

                   LINEARREG_ANGLE_14  LINEARREG_INTERCEPT_14  \
ticker date                                                     
TSLA   2024-08-06              -0.000                   0.000   
       2024-08-07              -0.000                   0.000   
       2024-08-08              -0.000                   0.000   
       2024-08-09              -0.000                   0.000   
       2024-08-12              -0.000                   0.000   

                   LINEARREG_SLOPE_14  ADXR_14      APO  AROONOSC_14  DEMA_30  \
ticker date                                                                     
TSLA   2024-08-06              -0.000  100.000 -164.218       -7.143   -0.000   
       2024-08-07              -0.000  100.000 -164.218       -7.143   -0.000   
       2024-08-08              -0.000  100.000 -164.218      -21.429   -0.000   
       2024-08-09              -0.000  100.000 -164.218      -21.429   -0.000   
       2024-08-12              -0.000  100.000 -164.218      -35.714   -0.000   

                   HT_TRENDLINE   MA_30  MIDPOINT_14  \
ticker date                                            
TSLA   2024-08-06         0.000 739.267        0.000   
       2024-08-07         0.000 739.267        0.000   
       2024-08-08         0.000 739.267        0.000   
       2024-08-09         0.000 739.267        0.000   
       2024-08-12         0.000 739.267        0.000   

         